In [64]:
from dataclasses import asdict
from pathlib import Path
import pickle

import fsspec
import mlflow
import pandas as pd

from nas import _REPO_ROOT
from nas.benchmark.nats_bench import ArchitectureResult

In [37]:
TRACKING_URI = str(_REPO_ROOT / "results" / "mlruns")
TRACKING_URI

'/home/tomaz/git/Politecnico/Extra/AI-Tech-Lab/nas/results/mlruns'

In [38]:
client = mlflow.tracking.MlflowClient(tracking_uri=TRACKING_URI)
client

In [9]:
selected_experiment = client.search_experiments(
    filter_string="name = 'sa_exponential_accuracy'"
)[0]
selected_experiment

<Experiment: artifact_location='file:///home/tomaz/git/Politecnico/Extra/AI-Tech-Lab/nas/results/mlruns/405556620328348245', creation_time=1719386971193, experiment_id='405556620328348245', last_update_time=1719386971193, lifecycle_stage='active', name='sa_exponential_accuracy', tags={}>

In [13]:
selected_runs = client.search_runs(selected_experiment.experiment_id)
len(selected_runs)

5

In [49]:
run_df = pd.json_normalize([dict(selected_run.data) for selected_run in selected_runs])
run_df

,metrics.accuracy,metrics.time,metrics.final_time,metrics.control_parameter,metrics.loss,metrics.final_time_per_epoch,metrics.final_loss,metrics.transition,metrics.time_per_epoch,metrics.final_accuracy,...,params.benchmark__path,params.optimiser__number_iterations,params.seed,params.optimiser__cooling_schedule__type,params.results_dir,tags.mlflow.source.git.commit,tags.mlflow.user,tags.mlflow.source.name,tags.mlflow.source.type,tags.mlflow.runName
0,83.944,46.063838,46.063838,0.135326,0.485844,3.838653,0.485844,0.0,3.838653,83.944,...,models/NATS-tss-v1_0-3ffb9-simple,100,4,exponential,/home/tomaz/git/Politecnico/Extra/AI-Tech-Lab/...,21cc657ad95bca2f23e6cedb9a5713f22ad4e6ab,tomaz,/home/tomaz/git/Politecnico/Extra/AI-Tech-Lab/...,LOCAL,serious-snake-588
1,84.504,46.302819,46.302819,0.135326,0.467763,3.858568,0.467763,0.0,3.858568,84.504,...,models/NATS-tss-v1_0-3ffb9-simple,100,3,exponential,/home/tomaz/git/Politecnico/Extra/AI-Tech-Lab/...,21cc657ad95bca2f23e6cedb9a5713f22ad4e6ab,tomaz,/home/tomaz/git/Politecnico/Extra/AI-Tech-Lab/...,LOCAL,enchanting-dog-35
2,84.746,56.301372,56.301372,0.135326,0.467404,4.691781,0.467404,1.0,4.691781,84.746,...,models/NATS-tss-v1_0-3ffb9-simple,100,2,exponential,/home/tomaz/git/Politecnico/Extra/AI-Tech-Lab/...,21cc657ad95bca2f23e6cedb9a5713f22ad4e6ab,tomaz,/home/tomaz/git/Politecnico/Extra/AI-Tech-Lab/...,LOCAL,likeable-bee-281
3,84.666,49.200553,49.200553,0.135326,0.468421,4.100046,0.468421,0.0,4.100046,84.666,...,models/NATS-tss-v1_0-3ffb9-simple,100,1,exponential,/home/tomaz/git/Politecnico/Extra/AI-Tech-Lab/...,21cc657ad95bca2f23e6cedb9a5713f22ad4e6ab,tomaz,/home/tomaz/git/Politecnico/Extra/AI-Tech-Lab/...,LOCAL,omniscient-jay-74
4,83.446,39.521631,39.521631,0.135326,0.492140,3.293469,0.492140,0.0,3.293469,83.446,...,models/NATS-tss-v1_0-3ffb9-simple,100,0,exponential,/home/tomaz/git/Politecnico/Extra/AI-Tech-Lab/...,21cc657ad95bca2f23e6cedb9a5713f22ad4e6ab,tomaz,/home/tomaz/git/Politecnico/Extra/AI-Tech-Lab/...,LOCAL,honorable-elk-36


In [54]:
selected_runs[0].info.artifact_uri

'file:///home/tomaz/git/Politecnico/Extra/AI-Tech-Lab/nas/results/mlruns/405556620328348245/131249662e7a440d8f5c3b7192fdf7ac/artifacts'

In [59]:
mlflow.artifacts.list_artifacts(
    selected_runs[0].info.artifact_uri, tracking_uri=TRACKING_URI
)

[<FileInfo: file_size=648, is_dir=False, path='artifacts/result.pkl'>]

In [62]:
run_results: list[ArchitectureResult] = []
for run in selected_runs:
    artifact_uri: str = run.info.artifact_uri
    with fsspec.open(Path(artifact_uri) / "result.pkl", "rb") as f:
        run_results.append(pickle.load(f))
run_results

[ArchitectureResult(index=5749, train=Metrics(loss=0.34832161025047303, accuracy=88.29599999145508, time_per_epoch=11.83192765712738, time=141.98313188552856), val=Metrics(loss=0.48584417125225066, accuracy=83.94400001220703, time_per_epoch=3.838653152897244, time=46.06383783476693), test=Metrics(loss=0.49638925375938414, accuracy=83.39, time_per_epoch=1.5354612611588976, time=18.42553513390677), flops=117.88353, size_parameters=0.830426, latency=0.018882371877369127),
 ArchitectureResult(index=10699, train=Metrics(loss=0.3077195804595947, accuracy=89.63799999877929, time_per_epoch=11.704105277856192, time=140.4492633342743), val=Metrics(loss=0.46776321168422696, accuracy=84.50399998535156, time_per_epoch=3.858568242618016, time=46.30281891141619), test=Metrics(loss=0.4689946113109589, accuracy=84.555, time_per_epoch=1.5434272970472065, time=18.521127564566477), flops=117.88353, size_parameters=0.830426, latency=0.01909610158518741),
 ArchitectureResult(index=9930, train=Metrics(loss=0

In [68]:
result_df = pd.json_normalize(asdict(result) for result in run_results)
result_df

,index,flops,size_parameters,latency,train.loss,train.accuracy,train.time_per_epoch,train.time,val.loss,val.accuracy,...,test.loss,test.accuracy,test.time_per_epoch,test.time,architecture.edge_0_to_1,architecture.edge_0_to_2,architecture.edge_1_to_2,architecture.edge_0_to_3,architecture.edge_1_to_3,architecture.edge_2_to_3
0,5749,117.88353,0.830426,0.018882,0.348322,88.296,11.831928,141.983132,0.485844,83.944,...,0.496389,83.390,1.535461,18.425535,3,1,3,2,3,1
1,10699,117.88353,0.830426,0.019096,0.307720,89.638,11.704105,140.449263,0.467763,84.504,...,0.468995,84.555,1.543427,18.521128,3,0,3,1,2,3
2,9930,184.73025,1.288506,0.020593,0.310551,89.502,14.672517,176.070208,0.467404,84.746,...,0.473220,84.480,1.876712,22.520549,3,3,3,1,3,3
3,1459,121.81569,0.858426,0.021500,0.311343,89.292,13.582911,162.994936,0.468421,84.666,...,0.474644,84.250,1.640018,19.680221,3,2,3,1,2,3
4,1000,82.49409,0.587386,0.018286,0.362608,87.542,9.545536,114.546432,0.492140,83.446,...,0.501785,83.595,1.317388,15.808652,1,3,2,0,1,3


In [70]:
result_df.describe()

,index,flops,size_parameters,latency,train.loss,train.accuracy,train.time_per_epoch,train.time,val.loss,val.accuracy,...,test.loss,test.accuracy,test.time_per_epoch,test.time,architecture.edge_0_to_1,architecture.edge_0_to_2,architecture.edge_1_to_2,architecture.edge_0_to_3,architecture.edge_1_to_3,architecture.edge_2_to_3
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.00000,5.000000
mean,5767.400000,124.961418,0.879034,0.019672,0.328109,88.854000,12.267400,147.208794,0.476315,84.26120,...,0.483007,84.054000,1.582601,18.991217,2.600000,1.80000,2.800000,1.000000,2.20000,2.600000
std,4553.533496,37.033349,0.253907,0.001329,0.025514,0.902651,1.964093,23.569112,0.011791,0.55295,...,0.014948,0.529733,0.202419,2.429023,0.894427,1.30384,0.447214,0.707107,0.83666,0.894427
min,1000.000000,82.494090,0.587386,0.018286,0.307720,87.542000,9.545536,114.546432,0.467404,83.44600,...,0.468995,83.390000,1.317388,15.808652,1.000000,0.00000,2.000000,0.000000,1.00000,1.000000
25%,1459.000000,117.883530,0.830426,0.018882,0.310551,88.296000,11.704105,140.449263,0.467763,83.94400,...,0.473220,83.595000,1.535461,18.425535,3.000000,1.00000,3.000000,1.000000,2.00000,3.000000
50%,5749.000000,117.883530,0.830426,0.019096,0.311343,89.292000,11.831928,141.983132,0.468421,84.50400,...,0.474644,84.250000,1.543427,18.521128,3.000000,2.00000,3.000000,1.000000,2.00000,3.000000
75%,9930.000000,121.815690,0.858426,0.020593,0.348322,89.502000,13.582911,162.994936,0.485844,84.66600,...,0.496389,84.480000,1.640018,19.680221,3.000000,3.00000,3.000000,1.000000,3.00000,3.000000
max,10699.000000,184.730250,1.288506,0.021500,0.362608,89.638000,14.672517,176.070208,0.492140,84.74600,...,0.501785,84.555000,1.876712,22.520549,3.000000,3.00000,3.000000,2.000000,3.00000,3.000000
